## Problem Statement 

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others 


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling 

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [1]:
import json 
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from pprint import pprint
from wordcloud import WordCloud, STOPWORDS


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import NMF
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.decomposition import NMF
from sklearn.model_selection import RepeatedStratifiedKFold

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading omw-1.4: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [WinError 10060] A connection attempt failed because
[nltk_data]     the connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>


## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [2]:
# Opening JSON file 
f = open('complaints-2021-05-14_08_16.json')# Write the path to your data file and load it 
  
# returns JSON object as  
# a dictionary 
data = json.load(f)
df=pd.json_normalize(data)

## Data preparation

In [3]:
# Inspect the dataframe to understand the given data.
df.head()

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,_source.consumer_disputed,_source.product,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,N/A,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,N/A,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I appreciate it if you could help me put a stop to Chase Bank cardmember services. \nIn 2018 I wrote to Chase asking for debt verification and what they sent me a statement which is not acceptable. I am asking the bank to validate the debt. Instead I been receiving mail every month from them attempting to collect a debt. \nI have a right to know this information as a consumer. \n\nChase account # XXXX XXXX XXXX XXXX Thanks in advance for your help.,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,N/A,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and was told by the agent who did the upgrade my anniversary date would not change. It turned the agent was giving me the wrong information in order to upgrade the account. XXXX changed my anniversary date from XX/XX/XXXX to XX/XX/XXXX without my consent! XXXX has the recording of the agent who was misled me.,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,N/A,Mortgage,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,N/A,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   _index                             78313 non-null  object 
 1   _type                              78313 non-null  object 
 2   _id                                78313 non-null  object 
 3   _score                             78313 non-null  float64
 4   _source.tags                       10900 non-null  object 
 5   _source.zip_code                   71556 non-null  object 
 6   _source.complaint_id               78313 non-null  object 
 7   _source.issue                      78313 non-null  object 
 8   _source.date_received              78313 non-null  object 
 9   _source.state                      76322 non-null  object 
 10  _source.consumer_disputed          78313 non-null  object 
 11  _source.product                    78313 non-null  obj

In [5]:
df.shape

(78313, 22)

In [6]:
#print the column names
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

In [7]:
#Assign new column names
df.columns = df.columns.str.replace('_source.', '')
df.columns = df.columns.str.replace('_', '')
df.columns

Index(['index', 'type', 'id', 'score', 'tags', 'zipcode', 'complaintid',
       'issue', 'datereceived', 'state', 'consumerdisputed', 'product',
       'companyresponse', 'company', 'submittedvia', 'datesenttocompany',
       'companypublicresponse', 'subproduct', 'timely',
       'complaintwhathappened', 'subissue', 'consumerconsentprovided'],
      dtype='object')

In [8]:
#Assign nan in place of blanks in the complaints column
df[df['complaintwhathappened'] == ''].index

Int64Index([    0,     3,     4,     5,     6,     7,     8,     9,    12,
               13,
            ...
            78292, 78298, 78299, 78300, 78302, 78304, 78305, 78306, 78307,
            78308],
           dtype='int64', length=57241)

There are 57241 rown in the complaints column with blanks, replacing it with nan.

In [9]:
df['complaintwhathappened'].replace('', np.nan, inplace = True)
df[df['complaintwhathappened'] == ''].index

Int64Index([], dtype='int64')

All blanks are replaced with NaN

In [10]:
df.shape

(78313, 22)

In [11]:
#Remove all rows where complaints column is nan
df = df[df['complaintwhathappened'].notna()]

In [12]:
df.shape

(21072, 22)

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [13]:
# Write your function here to clean the text and remove all the unnecessary elements.
# Making text lowercase
df['complaintwhathappened'] = df['complaintwhathappened'].str.lower()
# Removing the text in square brackets
df['complaintwhathappened'] = df['complaintwhathappened'].str.replace(r'[\[].*?[\]]','',regex=True)
#Removing punctuation
df['complaintwhathappened'] = df['complaintwhathappened'].str.replace('[^\(\)\w\s]','',regex= True)
#Removing words containing numbers
df['complaintwhathappened'] = df['complaintwhathappened'].str.replace('\w*\d\w*','',regex=True)

In [14]:
#Write your function to Lemmatize the texts
ticket = nltk.stem.WordNetLemmatizer()

def lemmatize(text):
    return " ".join([ticket.lemmatize(w) for w in word_tokenize(text)])

df['complaints_lemmatized'] = df.complaintwhathappened.apply(lemmatize)

In [15]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints 
df_clean = pd.DataFrame(df,columns=['complaintwhathappened','complaints_lemmatized'])

In [16]:
df_clean

,complaintwhathappened,complaints_lemmatized
1,good morning my name is xxxx xxxx and i appreciate it if you could help me put a stop to chase bank cardmember services \nin i wrote to chase asking for debt verification and what they sent me a statement which is not acceptable i am asking the bank to validate the debt instead i been receiving mail every month from them attempting to collect a debt \ni have a right to know this information as a consumer \n\nchase account xxxx xxxx xxxx xxxx thanks in advance for your help,good morning my name is xxxx xxxx and i appreciate it if you could help me put a stop to chase bank cardmember service in i wrote to chase asking for debt verification and what they sent me a statement which is not acceptable i am asking the bank to validate the debt instead i been receiving mail every month from them attempting to collect a debt i have a right to know this information a a consumer chase account xxxx xxxx xxxx xxxx thanks in advance for your help
2,i upgraded my xxxx xxxx card in and was told by the agent who did the upgrade my anniversary date would not change it turned the agent was giving me the wrong information in order to upgrade the account xxxx changed my anniversary date from xxxxxxxx to xxxxxxxx without my consent xxxx has the recording of the agent who was misled me,i upgraded my xxxx xxxx card in and wa told by the agent who did the upgrade my anniversary date would not change it turned the agent wa giving me the wrong information in order to upgrade the account xxxx changed my anniversary date from xxxxxxxx to xxxxxxxx without my consent xxxx ha the recording of the agent who wa misled me
10,chase card was reported on however fraudulent application have been submitted my identity without my consent to fraudulently obtain services do not extend credit without verifying the identity of the applicant,chase card wa reported on however fraudulent application have been submitted my identity without my consent to fraudulently obtain service do not extend credit without verifying the identity of the applicant
11,on while trying to book a xxxx xxxx ticket i came across an offer for to be applied towards the ticket if i applied for a rewards card i put in my information for the offer and within less than a minute was notified via the screen that a decision could not be made i immediately contacted xxxx and was referred to chase bank i then immediately contacted chase bank within no more than of getting the notification on the screen and i was told by the chase representative i spoke with that my application was denied but she could not state why i asked for more information about the xxxx offer and she explained that even if i had been approved the credit offer only gets applied after the first account statement and could not be used to purchase the ticket i then explicitly told her i was glad i got denied and i was absolutely no longer interested in the account i asked that the application be withdrawn and the representative obliged this all happened no later than after putting in the application on notwithstanding my explicit request not to proceed with the application and contrary to what i was told by the chase representative chase did in fact go ahead to open a credit account in my name on this is now being reported in my credit report and chase has refused to correct this information on my credit report even though they went ahead to process an application which i did not consent to and out of their error,on while trying to book a xxxx xxxx ticket i came across an offer for to be applied towards the ticket if i applied for a reward card i put in my information for the offer and within le than a minute wa notified via the screen that a decision could not be made i immediately contacted xxxx and wa referred to chase bank i then immediately contacted chase bank within no more than of getting the notification on the screen and i wa told by the chase representative i spoke with that my appli

In [ ]:
#Write your function to extract the POS tags 
def pos_tag(text):
  # write your code here
    text = text.split()
    tags = nltk.pos_tag(text)
    return " ".join([(word) for word, tag in tags if (tag=='NN')])

df_clean["complaint_POS_removed"] = df_clean['complaints_lemmatized'].apply(pos_tag) #this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].

In [ ]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [ ]:
# Write your code here to visualise the data according to the 'Complaint' character length
df_clean['complaint_length'] = df_clean['complaintwhathappened'].astype(str).apply(len)
df_clean = df_clean.sort_values(by = 'complaint_length', ascending = False)
df_clean

In [ ]:
# Plot the data according to character length of complaints
plt.figure(figsize=(10,6))
plt.hist(df_clean.complaint_length, bins = 30)
plt.title('Distribution of Character Length of Complaints', fontsize=25)
plt.ylabel('Number of Complaints', fontsize = 15)
plt.xlabel('Length of Characters in the Complaint', fontsize = 15)
plt.show()

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text
data = " ".join(review for review in df_clean.complaint_POS_removed)
wordcloud = WordCloud(max_words=40,background_color="white").generate(data)
plt.figure(figsize=[10,10])
plt.imshow(wordcloud,interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean). 
def get_top_n_words(corpus, n=None):
    data = " ".join(review for review in df_clean.complaint_POS_removed)
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words = get_top_n_words(df_clean['complaint_POS_removed'], 30)
    
df1 = pd.DataFrame(common_words, columns = ['unigram' , 'count'])
df1.groupby('unigram').sum()['count'].sort_values(ascending=False)

plt.figure(figsize=(15,6))
sns.barplot(x='unigram', y='count', data=df1, palette="coolwarm")
plt.xticks(rotation=90)
plt.title("Top 30 unigrams in the Complaint text", fontsize=20)
plt.show()

In [ ]:
#Print the top 10 words in the unigram frequency
common_words = get_top_n_words(df_clean['complaint_POS_removed'], 10)
for word, freq in common_words:
    print(word, freq)

In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean). 
def get_top_n_bigram(corpus, n=None):
    data = " ".join(review for review in df_clean.complaint_POS_removed)
    vec = CountVectorizer(ngram_range=(2,2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words = get_top_n_bigram(df_clean['complaint_POS_removed'], 30)
df1_bi = pd.DataFrame(common_words, columns = ['bigram' , 'count'])
df1_bi.groupby('bigram').sum()['count'].sort_values(ascending=False)

plt.figure(figsize=(15,6))
sns.barplot(x='bigram', y='count', data=df1_bi, palette="coolwarm")
plt.xticks(rotation=90)
plt.title("Top 30 bigrams in the Complaint text", fontsize=20)
plt.show()

In [ ]:
#Print the top 10 words in the bigram frequency
common_words = get_top_n_bigram(df_clean['complaint_POS_removed'], 10)
for word, freq in common_words:
    print(word, freq)

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean). 
def get_top_n_trigram(corpus, n=None):
    data = " ".join(review for review in df_clean.complaint_POS_removed)
    vec = CountVectorizer(ngram_range=(3,3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words = get_top_n_trigram(df_clean['complaint_POS_removed'], 30)
df1_tri = pd.DataFrame(common_words, columns = ['trigram' , 'count'])
df1_tri.groupby('trigram').sum()['count'].sort_values(ascending=False)

plt.figure(figsize=(15,6))
sns.barplot(x='trigram', y='count', data=df1_tri, palette="coolwarm")
plt.xticks(rotation=90)
plt.title("Top 30 trigrams in the Complaint text", fontsize=20)
plt.show()

In [ ]:
#Print the top 10 words in the trigram frequency
common_words = get_top_n_trigram(df_clean['complaint_POS_removed'], 10)
for word, freq in common_words:
    print(word, freq)

#### The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
#Write your code here to initialise the TfidfVectorizer 
tfidf = TfidfVectorizer(max_df = 0.95, min_df = 2, stop_words = 'english')

#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.
dtm = tfidf.fit_transform(df_clean['Complaint_clean'])

## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters 
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints 
* Correct the labels if needed 
* Map the clusters to topics/cluster names

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = 5 #write the value you want to test out

#keep the random_state =40
nmf_model = NMF(n_components=num_topics,random_state=40) #write your code here

In [ ]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names())

In [ ]:
#Print the Top15 words for each of the topics
W = nmf_model.fit_transform(dtm)  # Document-topic matrix
H = nmf_model.components_       # Topic-term matrix
words = np.array(tfidf.get_feature_names())
topic_words = pd.DataFrame(np.zeros((num_topics, 15)), index=[f'Topic {i + 1}' for i in range(num_topics)],
                           columns=[f'Word {i + 1}' for i in range(15)]).astype(str)
for i in range(num_topics):
    ix = H[i].argsort()[::-1][:15]
    topic_words.iloc[i] = words[ix]

topic_words

In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4
W.argmax(axis=1)

In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column
df_clean['Topic'] = W.argmax(axis=1) #write your code to assign topics to each rows.

In [ ]:
df_clean.head()

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean_5 = df_clean.groupby('Topic').head()
df_clean_5.sort_values('Topic')

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
#Create the dictionary of Topic names and Topics
Topic_names = {0:"Bank Account services",
               1:"Theft/Dispute Reporting", 
               2:"Others",
               3:"Credit card or prepaid card",
               4:"Mortgage/Loan"}

#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)


In [ ]:
df_clean

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
#Create the dictionary again of Topic names and Topics
Topic_names = {"Bank Account services":0,
               "Theft/Dispute Reporting":1,
               "Others":2,
               "Credit card or prepaid card":3,
               "Mortgage/Loan":4}

#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

In [ ]:
df_clean.shape

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data= df_clean[["complaintwhathappened","Topic"]]

In [ ]:
training_data

In [ ]:
training_data.isnull().sum()

#### Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [ ]:
#Write your code to get the Vector count
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data['complaintwhathappened'])
count_array = X_train_counts.toarray()
df_count = pd.DataFrame(data=count_array,columns = count_vect.get_feature_names())
df_count

In [ ]:
X_train_counts

In [ ]:
#Write your code here to transform the word vector to tf-idf
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_counts)

In [ ]:
#Creating Train and Test data
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.Topic, test_size = 0.25, random_state = 1)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [ ]:
# Creating a function to generate Classification Report
def metric_report(y_test, y_pred, model_name):
    # print classification report of classifier
    print(f"CLASSIFICATION REPORT for {model_name}\n")
    print(classification_report(y_test, y_pred, target_names=["Bank Account services", "Theft/Dispute Reporting", "Others", "Credit card or prepaid card",
"Mortgage/Loan"]))
    # plotting confusion matrix of the classifier
    plt.figure(figsize=(10,6))
    plt.title(f"CONFUSION MATRIX for {model_name}\n")
    matrix = confusion_matrix(y_test, y_pred)
    sns.heatmap(matrix, annot=True, cbar=None, cmap="Blues", fmt='d', xticklabels=["Bank Account services", "Theft/Dispute Reporting", "Others", "Credit card or prepaid card",
"Mortgage/Loan"], yticklabels=["Bank Account services", "Theft/Dispute Reporting", "Others", "Credit card or prepaid card",
"Mortgage/Loan"])
    plt.show()
    return

In [ ]:
# Write your code here to build any 3 models and evaluate them using the required metrics

<h3> Logistic Regression 

In [ ]:
logr = LogisticRegression(solver = 'liblinear')
%time
logr.fit(X_train,y_train)
y_pred_lr = logr.predict(X_test)
lr_acc = logr.score(X_test, y_test)
lr_f1 = f1_score(y_test,y_pred_lr,average ='weighted')
print("Logistic Regression Accuracy :",lr_acc)
print("Logistic Regression F1 Score :" , lr_f1)

In [ ]:
#Hyper Parameter Tuning
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2','l1']
c_values = [100, 10, 1.0, 0.1, 0.01]
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=logr, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)
grid_result = grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

In [ ]:
#Tuned Logistic Regression model
logr_tuned = LogisticRegression(solver='liblinear', C = 1.0, penalty='l1',random_state=1)
%time
logr_tuned.fit(X_train,y_train)
y_pred_lr_tuned = logr_tuned.predict(X_test)
lr_acc_tuned = logr_tuned.score(X_test, y_test)
lr_f1_tuned = f1_score(y_test,y_pred_lr_tuned,average ='weighted')
print("Logistic Regression Accuracy after tuning:",lr_score_tune)
print("Logistic Regression F1 Score after tuning:" , lr_f1_tuned)

In [ ]:
#Logistic Regression Metric Report
metric_report(y_test,y_pred_lr_tuned,logr_tuned)

### Decision Tree

In [ ]:
dtree = DecisionTreeClassifier()
%time
dtree.fit(X_train,y_train)
y_pred_dt = dtree.predict(X_test)
dt_acc = dtree.score(X_test, y_test)
dt_f1 = f1_score(y_test,y_pred_dt,average ='weighted')
print("Decision Tree Accuracy:",dt_acc)
print("Decision Tree F1 Score :" , dt_f1)

In [ ]:
# Hyperparameter tuning
dtree_params = {
     'criterion': ['gini', 'entropy'],
     'max_depth' : [5, 10, 15, 20, 25, 30],
     'min_samples_leaf':[1,5,10,15, 20, 25],
     'max_features':['auto','log2','sqrt', None]}

dtree_grid = GridSearchCV(estimator=dtree, param_grid=dtree_params, verbose=1, scoring='f1_weighted', n_jobs=-1, cv= 5)
dtree_grid.fit(X_train, y_train)
print(dtree_grid.best_params_)

In [ ]:
# Decision Tree after hyper paramater tuning
dtree_tuned = DecisionTreeClassifier(random_state=1, criterion='gini', min_samples_leaf= 15, max_features= None, max_depth= 25)
%time
dtree_tuned.fit(X_train,y_train)
y_pred_dt_tuned = dtree_tuned.predict(X_test)
dt_acc_tuned = dtree_tuned.score(X_test, y_test)
dt_f1_tuned= f1_score(y_test,y_pred_dt_tuned,average ='weighted')
print("Decision Tree Accuracy after tuning:",dt_acc_tuned)
print("Decision Tree F1 Score after tuning:" , dt_f1_tuned)

In [ ]:
# Decsition Tree Metric Report
metric_report(y_test,y_pred_dt_tuned,dtree_tuned)

### Random Forest

In [ ]:
rf = RandomForestClassifier(random_state= 1)
%time
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
rf_acc = rf.score(X_test, y_test)
rf_f1 = f1_score(y_test,y_pred_rf,average ='weighted')
print("Random Forest Accuracy:", rf_acc)
print("Random Forest F1 Score :" , rf_f1)

In [ ]:
# Hyper Paramater Tuning
params ={'max_depth': [10,20, 30, 40, 50, 100], 'max_features': ['auto', 'sqrt', None], 'min_samples_leaf': [1, 2, 3, 4, 5], 'min_samples_split': [2, 5, 10],'n_estimators': [200, 500, 1000, 1500, 2000]}
rf_rand = RandomizedSearchCV(estimator = rf, param_distributions = params, cv = 5, scoring = "f1_weighted", n_jobs = -1, random_state = 1)
rf_rand.fit(X_train, y_train)
print(rf_rand.best_params_)

In [ ]:
# Random Forest after using hyper paramater tuning
rf_tuned = RandomForestClassifier(random_state= 1, n_estimators= 1500, min_samples_split= 2, min_samples_leaf= 1, max_features= 'sqrt', max_depth= 100)
%time
rf_tuned.fit(X_train,y_train)
y_pred_rf_tuned = rf_tuned.predict(X_test)
rf_acc_tuned = rf_tuned.score(X_test, y_test)
rf_f1_tuned = f1_score(y_test,y_pred_rf_tuned,average ='weighted')
print("Random Forest Accuracy after tuning:", rf_acc_tuned)
print("Random Forest F1 Score after tuning:" , rf_f1_tuned)

In [ ]:
# Random Forest Metric Report
metric_report(y_test,y_pred_rf_tuned, rf_tuned)

In [ ]:
# Creating a summary of perfromance of all models and comparing
summary = pd.DataFrame({'Model': ['Logistic Regression'], 'Accuracy': [round(lr_acc_tuned, 2)], 'F1 Score': [round(lr_f1_tuned, 2)]})
summary = summary.append({'Model': 'Decision Tree', 'Accuracy': round(dt_acc_tuned, 2) , "F1 Score": round(dt_f1_tuned, 2) }, ignore_index=True)
summary = summary.append({'Model': 'Random Forest', 'Accuracy': round(rf_acc_tuned, 2) , "F1 Score": round(rf_f1_tuned, 2) }, ignore_index=True)
summary

From the above table it's clear that <b> Logistic Regression </b> is the best model. Thus using it for ticket class prediction

In [ ]:
#Defining the customized function to use for ticket class prediction
def custom_preprocessor(text):
    return ' '.join(text).lower()

def ticket_class_prediction(text):
    target_names = ["Bank Account services", "Theft/Dispute Reporting", "Others", "Credit card or prepaid card", "Mortgage/Loan"]
    vectorizer = count_vect
    transformer = TfidfTransformer()
    X_train_tfidf = transformer.fit_transform(X_train_counts)
    model = logr_tuned
    preprocessed_text = custom_preprocessor(text)
    X_test_counts = vectorizer.transform([preprocessed_text])
    X_test_tfidf = transformer.transform(X_test_counts)
    predicted = model.predict(X_test_tfidf)
    return(target_names[predicted[0]])

In [ ]:
# Testing with custom input
text = ['i have a dispute with the customer service']
text1 = ['My credit card is not working']
text2 = ['My loan is not yet sanctioned']
text3 = ['I would like to change personal information in my account']
print(" Ticket belongs to the class ", ticket_class_prediction(text))
print(" Ticket belongs to the class ", ticket_class_prediction(text1))
print(" Ticket belongs to the class ", ticket_class_prediction(text2))
print(" Ticket belongs to the class ", ticket_class_prediction(text3))